# Transformation of CP to renta media

We use data from https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfCodPostal/2016/jrubikf15b9305df2e5d53b0bbd20afaea102233fc84fd9.html that contains data of the renta media disponible per CP

In [1]:
import pandas as pd
import re
from unidecode import unidecode

In [2]:
renta_media = pd.read_excel('../data/raw/renta_cp.xlsx')
final_merge = pd.read_csv('../data/raw/final_merge.csv')

In [3]:
def remove_accents(text):
    return unidecode(text)

In [4]:
renta_media.columns = map(remove_accents,renta_media.columns)
renta_media.columns = [col.replace(' ', '_').lower() for col in renta_media.columns]

In [5]:
renta_media.head(13)

,region,numero_de_declaraciones,renta_bruta_media,renta_disponible_media
0,Jerez de la Frontera-11020,75867,23037,19415
1,11400-Las Pachecas-El Mojo-Los Isletes,1165,17956,15427
2,11401-Sur,4685,22476,19017
3,11402-Centro,3266,29740,24292
4,11403-Centro-Catedral San Salvador-Alcázar,2273,26173,21552
5,11404-Hospital de Jerez,4444,19550,16987
6,11405-Norte-Noreste-Granja,16121,25673,21249
7,11406-Este-Delicias,15434,21360,18299
8,11407-Centro,10510,28486,23359
9,11408-Oeste-Centro,10479,19528,16902


In [6]:
#First we want to extract the CPs of the "Código postal" column. We will use regex.
data = renta_media['region']
renta_media['codigo_postal'] = None

for index, address in enumerate(renta_media['region']):  #enumerate() keeps track of index and value
    match = re.search(r'\b\d{5}\b', address)     # Using regex to find a sequence of 5 digits
    
    if match:
        renta_media.loc[index, 'codigo_postal'] = match.group() #group() se utiliza para extraer el match
        
    else:
        renta_media.loc[index, 'codigo_postal'] = None

In [7]:
renta_media.columns

Index(['region', 'numero_de_declaraciones', 'renta_bruta_media',
       'renta_disponible_media', 'codigo_postal'],
      dtype='object')

In [8]:
#from all columns we are interested in codigo_postal and renta_disponible_media
renta_media = renta_media[['codigo_postal', 'renta_disponible_media']]

In [9]:
renta_media = renta_media.dropna(subset=['codigo_postal'])

In [11]:
renta_media['codigo_postal'] = renta_media['codigo_postal'].astype(int)

In [76]:
renta_media['codigo_postal'].value_counts()

codigo_postal
28005    2
3014     2
8015     2
8019     2
28006    2
        ..
35013    1
35012    1
35011    1
35010    1
46112    1
Name: count, Length: 564, dtype: int64

In [13]:
#Now, let's check if we can match CP of students residence with the renta_disponible_media
final_merge.columns

Index(['NIA', 'curs_preinscripcio', 'convocatoria', 'curs_matricula', 'sexe',
       'data_naixement', 'nacionalitat', 'pais', 'CP', 'poblacio', 'comarca',
       'provincia', 'via_acces', 'nota_acces', 'ordre_assignacio',
       'orientacio', 'aspectes_eleccio', 'any_qualificacio',
       'convocatoria_qualificacio', 'treball_remunerat', 'estudis_pare',
       'estudis_mare', 'ocupacio_pare', 'ocupacio_mare',
       'codi_centre_secundaria', 'any_acabament_secundaria',
       'tipus_centre_secundaria', 'tipus_lloc_secundaria',
       'codi_lloc_secundaria', 'estudis_universitaris',
       'any_acces_primera_vegada_universitat', 'titulacio_nivell_assolit',
       'modalitat_acces', 'codi_grau', 'pla', 'any_academic', 'assignatura',
       'codi_assignatura', 'grup', 'tipus', 'semestre', 'exhaurides',
       'superada', 'nota_numerica', 'nota_alfa'],
      dtype='object')

In [21]:
final_merge_students = final_merge.groupby('NIA').first().reset_index()

In [69]:
#cp_students = final_merge_students[['CP']].copy()
final_merge_students['CP'] = final_merge_students['CP'].astype(str)
final_merge_students['CP'] = final_merge_students['CP'].apply(lambda x: int('0' + str(x)) if len(str(x)) == 4 else int(x))

In [70]:
final_merge_students['CP']

0        8901
1        8917
2        8034
3        8391
4        8820
        ...  
1302    25199
1303     8017
1304     8320
1305    12580
1306     8014
Name: CP, Length: 1307, dtype: int64

In [53]:
merged_df = pd.merge(final_merge_students, renta_media, left_on=['CP'], right_on=['codigo_postal'], how='left',indicator=True)

In [55]:
final_merge_students.shape

(1307, 45)

In [56]:
only_in_students = merged_df[merged_df['_merge'] == 'left_only'].copy()
only_in_students

,NIA,curs_preinscripcio,convocatoria,curs_matricula,sexe,data_naixement,nacionalitat,pais,CP,poblacio,...,grup,tipus,semestre,exhaurides,superada,nota_numerica,nota_alfa,codigo_postal,renta_disponible_media,_merge
3,1156275,2017,J,2017,H,1983-03-04,ESP,ESP,8391,Tiana,...,801,B,1,1,S,7.6,NT,NaN,NaN,left_only
4,1157938,2016,J,2016,H,1979-07-10,ESP,ESP,8820,"Prat de Llobregat, El",...,801,B,1,1,N,0.0,NV,NaN,NaN,left_only
7,1176665,2014,J,2014,H,1987-09-15,ESP,ESP,8770,Sant Sadurni d'Anoia,...,201,B,1,1,N,0.0,NP,NaN,NaN,left_only
9,1195977,2014,S,2014,H,1980-10-23,ESP,ESP,8820,"Prat de Llobregat, El",...,701,B,2,2,N,0.0,NP,NaN,NaN,left_only
11,1211590,2015,J,2015,D,1976-06-24,ESP,ESP,8191,Rubi,...,801,B,1,1,S,5.0,AP,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1687112,2022,F,2022,D,2002-10-24,ESP,ESP,8197,Sant Cugat del Valles,...,601,B,2,1,S,9.3,M,NaN,NaN,left_only
1321,1687116,2022,F,2022,H,2003-06-30,ESP,ESP,7500,Manacor,...,801,O,2,1,S,7.1,NT,NaN,NaN,left_only
1322,1687120,2022,F,2022,H,2003-04-30,ESP,ESP,25199,Lleida,...,501,B,2,1,S,7.0,NT,NaN,NaN,left_only
1324,2116062,2015,S,2015,H,1983-05-01,ESP,ESP,8320,"Masnou, El",...,701,O,1,1,S,10.0,M,NaN,NaN,left_only
